In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['font.family'] = 'monospace'
#rcParams['font.sans-serif'] = ['Tahoma']

from glob import glob
import math
import time
import k2flix
from matplotlib import rc

rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
#-----------------------------------10-------------------------------------

def display_overlay(oid_list,location, start, stop):

    #os.chdir(location)
    all_flux={}
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for idx, oid in enumerate(oid_list):
        flux = pd.read_csv('{}/{}_flux.csv'.format(location,oid),
                          usecols = ['flux_gap','time','error'])
        s = start
        t = stop
        mean = flux.flux_gap.median()
        plt.plot(list(flux.time)[s:t],list(flux.flux_gap/mean)[s:t])
        plt.xlabel('Time (BJD - 2454833 days)')
        plt.ylabel(r'Flux (e- sec$^{-1}$)')
        plt.legend()
    plt.savefig('stars_shortlist/share/{}_{}_{}.png'.format(test,start,stop),dpi=300)
    plt.show()
    return
    
def edges_to_bool(start_stop):
    
    if start_stop.empty:
        return [0]
    else:
        rangeflare = []
        for i,row in start_stop.iterrows():
                rangeflare.append(list(range(int(row.istart),int(row.istop))))
        isflare = np.zeros(start_stop.istop.iloc[-1]+1)
        rangeflare = [item for sublist in rangeflare for item in sublist]
        np.put(isflare,rangeflare,np.ones_like(rangeflare))
        return isflare

def create_inflated_binoms(oid_list,p,maxlen,display=False):

    inflated_binoms={}
    for i in [1,3,5]:
        binom = 0
        for oid in oid_list:
            not_inflated = np.random.binomial(1,p[oid]/i,maxlen//i)
            inflated = [item for sublist in [[j]*i for j in not_inflated] for item in sublist]
            binom = np.add(inflated,binom)
        inflated_binoms[i]=binom
    if display == True:
        plt.plot(inflated_binoms[3])
    
    return inflated_binoms

def remove_systematics(seq,maxlen,maxpoints=5):
    
    #create cleaned data frame:
    systematics = list(seq[maxpoints<seq.sum(axis=1)].index)
    remove = sorted(list(set([item 
                              for sublist in [[i-1,i,i+1] 
                                              for i in systematics] 
                              for item in sublist])))
    seq_drop = seq.drop(labels=remove)
    
    #overlap dirty, overlap clean:
    od = np.asarray([seq[seq.sum(axis=1)>=i].shape[0] for i in range(11)])
    oc = np.asarray([seq_drop[seq_drop.sum(axis=1)>=i].shape[0] for i in range(11)])

    #Calculate the binomial probability for cleaned version
    p = seq.sum()/maxlen
    return od, oc, p, remove

def display_comparison(inflated_binoms, overlap_dirty,overlap_clean, test,cluster):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
    lwd=3
    for i,binom in inflated_binoms.items():
        overlap_binom, edges = np.histogram(binom, bins=11, range=(0,10))
        #print(overlap_binom)
        overlap_binom = np.cumsum(overlap_binom[::-1])[::-1]
        #print(overlap_binom,'\n',overlap)
        ax.plot(overlap_binom,
                label='{}-data points binomial distribution'.format(i),
                linewidth=lwd)
    ax.plot(overlap_dirty,
            label='Real distribution with systematic errors',
            c='k',linewidth=lwd)
    ax.plot(overlap_clean,
            label='Real distribution - systematics removed',
            c='k',linestyle='dashed',linewidth=lwd)
   
    #ax.plot(np.array(overlap)-overlap_binom,label='real-random')
    ax.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=1.)
    ax.set_yscale('log')
    plt.xlabel('\# simultaneous FC data points',fontsize=12)
    plt.ylabel('\# data points ( with $\leq$ \# simultaneous FC data points )',fontsize=12)
    plt.xlim(0,10)
    print(os.getcwd())
    plt.savefig('stars_shortlist/share/{}_{}_sys_cuts.png'.format(cluster,test),dpi=300)
    fig.show()
    return

def generate_sequence(oid_list,location):

    #os.chdir(location)

    all_isflare = {}
    rem = []
    for oid in oid_list:
        try:
      
            start_stop = pd.read_csv('{}/{}_flares.csv'.format(location,oid))
            all_isflare[oid] = edges_to_bool(start_stop)
        except FileNotFoundError:
            rem.append(oid)
            continue
    maxlen = max([len(val) for key,val in all_isflare.items()])
    setA = set(oid_list)
    # Get new set with elements that are only in a but not in b
    oid_list = list(setA.difference(rem))
    for key, val in all_isflare.items():
        all_isflare[key] = np.concatenate((val,np.zeros(maxlen-len(val))))
        #print(len(val))
    seq = pd.DataFrame(all_isflare)
    return oid_list, maxlen, seq

def systematic_set(remove):
    '''
    Takes a boolean array of indices to remove and converts them to a list of edges to continuous sets of indices.
    
    Input:
    
    remove - boolean array of indices into flux
    
    Return:
    
    s - Series of 
    
    '''
    s=[]
    j = 0
    for i, item in enumerate(remove[:-1]):
            if item+1 != remove[i+1]:
                s.append(remove[j:i+1])
                j = i+1 
    return s

def systematics_wrap(location, oid_list, test, cluster, maxpoints=5, display=False):
       
    '''
    
    Find systematic errors that can be identified from correlations between LCs in a single campaign 
    and write the indices into flux that need to be removed according to it in a file.
    
    Input:
    
    location - path to data
    oid_list - list of EPIC IDs that have LCs and TargetPixelFiles in the same campaign
    display, save - flags for figure outputs
    
    '''
 
    oid_list, maxlen, seq = generate_sequence(oid_list, location)
    overlap_dirty, overlap_clean, p, remove = remove_systematics(seq,maxlen,maxpoints=maxpoints)
    sys_set = pd.Series([i for item in systematic_set(remove) for i in item])
    inflated_binoms = create_inflated_binoms(oid_list, p, maxlen,display=display)
    sys_set.to_csv('stars_shortlist/{0}/results/{1}/results/{0}_systematics.csv'.format(cluster,test))
    if display == True:
        display_comparison(inflated_binoms, overlap_dirty, overlap_clean, test,cluster)
        try:
            sys_s = sys_set.drop(sys_set.index[-1])
            j=0
            for i,item in sys_s.iteritems():
                if sys_set[i+1]-sys_set[i] > 1:
                    j+=1
        except IndexError:
            print('No systematics found')
            pass

    return oid_list, sys_set, overlap_dirty


#----------------------------------11-------------------------------------


def find_nearest(array,value):
    '''
    
    Helper function that performs a binary search for the closest value in an array:
    
    Input:
    
    array - sorted array
    value - same type as elements in array
    
    Return:
    
    closest element in array
    index of the closest element in array
    
    '''

    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1], idx-1
    else:
        return array[idx], idx


def tpf_time_match(flux, tpf):

    flags = []
    timed = []
    quality = tpf.hdulist[1].data['QUALITY']
    print(time.clock())
    for id_,row in flux.iterrows():

        t, idx_ = find_nearest(tpf.bkjd(), row.time)
        timed.append(t)
        flags.append(quality[idx_])

    flux['tpf_flags'] = flags
    flux['tpf_time'] = timed
    return flux



def parallel_wrap(tpfs, flux, flares,oid,cluster,test,C):
    print(oid)
    try:
        tpfs[oid] = k2flix.TargetPixelFile('stars_shortlist/{}/ktwo{}-c{}_lpd-targ.fits.gz'.format(cluster,oid,C))
    except FileNotFoundError:
        os.chdir('/work1/eilin/appaloosa/appaloosa/stars_shortlist/{}'.format(cluster))
        command = 'wget -q http://archive.stsci.edu/missions/k2/target_pixel_files/c{}/{}00000/{}000/ktwo{}-c{}_lpd-targ.fits.gz'.format(C[1],str(oid)[:4],str(oid)[4:6],oid,C)
        print(command)
        os.system(command)
        print(os.getcwd())
        tpfs[oid] = k2flix.TargetPixelFile('ktwo{}-c{}_lpd-targ.fits.gz'.format(oid,C),cache=False)
        os.chdir('/work1/eilin/appaloosa/appaloosa')
    try:
        flares[oid] = pd.read_csv('stars_shortlist/{}/results/{}/results/{}_flares.csv'.format(cluster,test,oid))

        if glob('stars_shortlist/{}/results/{}/results/{}_tpf_times_FLUX.csv'.format(cluster,test,oid)) == []:
            flux[oid] = pd.read_csv('stars_shortlist/{}/results/{}/results/{}_flux.csv'.format(cluster,test,oid),usecols=['error','flux_gap','time','flux_model'])
            flux[oid] = tpf_time_match(flux[oid],tpfs[oid])
            flux[oid].to_csv('stars_shortlist/{}/results/{}/results/{}_tpf_times_FLUX.csv'.format(cluster,test,oid))
        else:
            flux[oid] = pd.read_csv('stars_shortlist/{}/results/{}/results/{}_tpf_times_FLUX.csv'.format(cluster,test,oid),
                                usecols=['error','flux_gap','time','flux_model','tpf_time','tpf_flags'])
    except FileNotFoundError:
        pass
    print(len(tpfs))
    return tpfs, flux, flares

def wrap_tpf_time_match(oid_list,cluster,test,C):
    tpfs, flux, flares = dict(), dict(), dict()
    for oid in oid_list:
        parallel_wrap(tpfs, flux, flares,oid,cluster,test,C)
    #num_cores = multiprocessing.cpu_count()
    #print('Number of cores: {}'.format(num_cores))
    #Parallel(n_jobs=num_cores-3)(delayed(parallel_wrap)(tpfs,flux,flares,oid,cluster,test,C) for oid in oid_list )

    return flux, flares, tpfs
                                          
def edges_to_bool2(start_stop,flux):
    
    rangeflare = []
    for i,row in start_stop.iterrows():
            rangeflare.append(list(range(int(row.istart),int(row.istop+1))))
    
    isflare = np.zeros_like(flux.time.tolist())
    rangeflare = pd.Series([item for sublist in rangeflare for item in sublist], 
                           dtype=np.int)
    rangeflare = rangeflare[rangeflare < isflare.shape[0]]
    np.put(isflare,rangeflare,np.ones_like(rangeflare))

    return isflare  

def bool_to_edges(isflare):
    
    c, start, stop = False, [], []
    isflare = list(isflare)
    for id_, bool_ in enumerate(isflare):
        if c == False and bool_ == True:
            start.append(id_)
            c = True
        elif c == True and bool_ == False:
            stop.append(id_-1)
            c = False
    start_stop = pd.DataFrame({'istart':start},dtype=int)#,'istop':stop})
    start_stop = start_stop.join(pd.Series(stop,name='istop',dtype=int))
    return start_stop   




def flag_df(flux):
    index = [1,8192,524288]
    columns = ['>',
               '>_after_thruster_removal',
               '>_after_systematics_removal',
               '>_left_overall',
               '>_coinciding_with_flare_candidates',
               '>_coinciding_with_clean_flare_candidates',
              ]
    flags = pd.DataFrame(index=index, columns=columns)
    for flag in index:
        flags[columns[0]].loc[flag] = flux[flux.tpf_flags >= flag].shape[0]
        flags[columns[1]].loc[flag] = flux[flux.new_tpf_flags >= flag].shape[0]
        flags[columns[2]].loc[flag] = flux[flux.tpf_flags_wo_systematics >= flag].shape[0]
        flags[columns[3]].loc[flag] = flux[(flux.tpf_flags_wo_systematics >= flag) & (flux.new_tpf_flags >= flag)].shape[0]
        flags[columns[4]].loc[flag] = flux[(flux.isflare == True) & (flux.tpf_flags >= flag)].shape[0]
        flags[columns[5]].loc[flag] = flux[(flux.isflare_no_sys_no_thruster == True) & (flux.tpf_flags >= flag)].shape[0]
    return flags.T

def flagdecomp(x,decomp=[]):
    '''
    Decompose cumulative flags
    '''
    flags = [0,1] 
    for i in range(1,20):
        flags.append(flags[i]*2)
        
    flag,i = find_nearest(flags, x)
    
    if x == 0:
        return decomp
    elif flag > x:
        decomp.append(flags[i-1])
        return flagdecomp(x-flags[i-1],decomp)
    elif flag <= x:
        decomp.append(flag)
        return flagdecomp(x-flag, decomp)
    return decomp


In [11]:

cluster = 'M67'
C = '05'
test = 'run_06'
#---------------------------
#o = 211412571 #M67
#o = 211954033 #M44
#o = 247111108 #NGC_1647
#------------------------
os.chdir('/work1/eilin/appaloosa/appaloosa')
print(os.getcwd())
params = pd.read_csv('stars_shortlist/{0}/results/{1}/catalog/{0}_parameter.csv'.format(cluster,test))
params = params[params.todrop == False]
oid_list = params.EPIC.astype(int).tolist()

#oid_list.remove(211073598)

/work1/eilin/appaloosa/appaloosa


In [12]:
loc = 'stars_shortlist/{}/results/{}/results'.format(cluster,test)
oid_list, sys, overlap = systematics_wrap(loc, oid_list, test, cluster, maxpoints=6, display=True)

OSError: File b'stars_shortlist/M67/results/run_06/results/211393067_flares.csv' does not exist

In [5]:
try:
    sys_s = pd.read_csv('stars_shortlist/{0}/results/{1}/results/{0}_systematics.csv'.format(cluster,test),
                   header=None,usecols=[1])
    print(sys_s.shape)
except ValueError:
    print('No systematics.')


(31, 1)


## Match target pixel files with light curve time series

In [6]:
os.chdir('/work1/eilin/appaloosa/appaloosa')
#oid_list.remove(211123901)
#os.chdir('stars_shortlist/{}'.format(cluster))
flux, flares, tpfs = wrap_tpf_time_match(oid_list, cluster, test, C)

211101696
1
211021827
2
211038211
3
211025925
4
210964488
5
210882573
6
211038222
7
210888723
8
211079188
9
211060762
10
211142685
11
211154974
12
211030057
13
211054634
14
211105839
15
210880560
16
211083317
17
210817078
18
211101750
19
211030074
20
211122237
21
211062846
22
211142730
23
211093581
24
211091536
25
210817106
26
211007577
27
211040347
28
211046494
29
211089510
30
211085419
31
211177579
32
211013743
33
211105906
34
211103863
35
210860152
36
210950265
37
211171449
38
211073154
39
210978953
40
210915466
41
210995339
42
210862226
43
210944157
44
211079332
45
211026087
46
211042471
47
211056809
48
210878635
49
210966700
50
210974893
51
211038382
52
211065006
53
211103923
54
210983092
55
211038389
56
210872505
57
210960579
58
211015877
59
211036360
60
211093705
61
210917578
62
210854098
63
210833622
64
211034329
65
211138781
66
210940129
67
211056865
68
211103969
69
211007717
70
211077349
71
211034343
72
211060969
73
211112171
74
211065077
75
211052796
76
210872577
77
21106304

594
211068579
595
211062438
596
211107496
597
210919084
598
211095218
599
210996915
600
211048116
601
211111611
602
211005117
603
211048126
604
211017407
605
211123901
606
211115713
607
211103431
608
211070664
609
211058381
610
211064526
611
211132111
612
211093202
613
211093203
614
211046102
615
211095259
616
211136223
617
211113696
618
210927331
619
211134185
620
211005163
621
211064559
622
211074799
623
210894583
624
210994935
625
211089146
626
211095292
627
211017469
628
211029757
629
211042045
630
211003154
631
210994964
632
211046168
633
211056415
634
211035936
635
211052320
636
211085091
637
211058468
638
211107623
639
211074858
640
211025708
641
211019565
642
211027758
643
210886447
644
211060530
645
211128116
646
210947895
647
211052343
648
210939705
649
211097402
650
211124025
651
211142460
652
211130173
653
211027775
654
210995008
655
211035968
656
211072835
657
211023687
658
211074888
659
211132233
660
211062604
661
211033935
662
211078992
663
210837336
664
210978650
665
21

In [ ]:
from astropy.io import fits
f = fits.open('stars_shortlist/Pleiades/ktwo211123901-c04_lpd-targ.fits.gz')
f.info()

### Remove thruster firings

In [8]:

def remove_thruster_firings(tpfs, flux, flares, oid):
    remove = []
    thr = []
    for i in range(len(tpfs[oid].hdulist[1].data['FLUX'])): 
        if tpfs[oid].hdulist[1].data['QUALITY'][i] > 545288:#flag values are summed up!
            thr.append(i)
            remove.append(tpfs[oid].bkjd(i))
            try:
                remove.append(tpfs[oid].bkjd(i-1))
            except IndexError:
                print('IndexError -1')
                pass
            try:
                remove.append(tpfs[oid].bkjd(i+1))
            except IndexError:
                print('IndexError +1')
                pass
    remove_id = []
    for rt in sorted(list(set(remove))):
        rem_id = flux[oid].index.values[np.round(flux[oid].time,6) == np.round(rt,6)]
        if rem_id != []:
            remove_id.append(rem_id)

    isflare = edges_to_bool2(flares[oid],flux[oid])
    new_isflare = np.array(isflare)
    new_flags = np.array(flux[oid].tpf_flags)
    for id_ in remove_id:
        if isflare[id_] == 1.:
            for j, row in flares[oid].iterrows():
                if (row.istart <= id_) & (row.istop+1 >=id_):
                    new_isflare[row.istart:row.istop+1] = 0
                    new_flags[row.istart:row.istop+1] = 0

    flux[oid]['isflare'] = isflare
    flux[oid]['new_isflare'] = new_isflare
    flux[oid]['new_tpf_flags'] = new_flags
    return thr


def remove_systematics2(flux,test,cluster,oid):

    s = pd.read_csv('stars_shortlist/{1}/results/{0}/results/{1}_systematics.csv'.format(test,cluster),names=['systematics']).systematics.tolist()
    flux[oid]['tpf_flags_wo_systematics'] = np.array(flux[oid].tpf_flags)
    flux[oid]['isflare_wo_systematics'] = np.array(flux[oid]['isflare'])
    flux[oid]['tpf_flags_wo_systematics'].iloc[s]= 0
    flux[oid]['isflare_wo_systematics'].iloc[s]= 0
    return

def rtf(tpfs,flux,flares,test,cluster,oid):
    print(oid)
    dp = remove_thruster_firings(tpfs, flux, flares, oid)
    remove_systematics2(flux, test, cluster, oid)
    
    flux[oid]['isflare_no_sys_no_thruster'] = (flux[oid].new_isflare == 1.) & (flux[oid].isflare_wo_systematics == 1.)
    
    start_stop = bool_to_edges(flux[oid].isflare_no_sys_no_thruster)
    flares[oid] = flares[oid].join(start_stop,rsuffix='_no_sys_no_thruster')
    
    #flux[oid].to_csv('results/{}/{}_tpf_times_FLUX.csv'.format(test,oid))

    flares[oid].to_csv('stars_shortlist/{}/results/{}/results/{}_flares.csv'.format(cluster,test,oid))
    flux[oid].to_csv('stars_shortlist/{}/results/{}/results/{}_flux.csv'.format(cluster,test,oid))
    return dp


#oid_list.remove(211066477)
for oid in oid_list:
    dp = rtf(tpfs,flux,flares,test,cluster,oid)


211101696
211021827
211038211
211025925
210964488
210882573
211038222
210888723
211079188
211060762
211142685
211154974
211030057
211054634
211105839
210880560
211083317
210817078
211101750
211030074
211122237
211062846
211142730
211093581
211091536
210817106
211007577
211040347
211046494
211089510
211085419
211177579
211013743
211105906
211103863
210860152
210950265
211171449
211073154
210978953
210915466
210995339
210862226
210944157
211079332
211026087
211042471
211056809
210878635
210966700
210974893
211038382
211065006
211103923
210983092
211038389
210872505
210960579
211015877
211036360
211093705
210917578
210854098
210833622
211034329
211138781
210940129
211056865
211103969
211007717
211077349
211034343
211060969
211112171
211065077
211052796
210872577
211063049
211005706
210905362
210997527
210983198
211024158
211108136
211063081
211132712
210964780
210977069
211101996
210866482
210907442
211091768
211032382
211059010
211138883
211011910
210901321
211065162
211114317
211118414


In [10]:
dp = rtf(tpfs,flux,flares,test,cluster,oid_list[0])
s = len(set(sys_s[1].tolist()+dp))
print(len(dp),len(sys_s[1].tolist()),s,sys_s[1].tolist()+dp)
sum([(flux[oid].shape[0]-s)*29.4/60./24./365.25 for oid in oid_list])

211101696
166 31 196 [140, 141, 142, 602, 603, 604, 605, 785, 786, 787, 1157, 1158, 1159, 1229, 1230, 1231, 1286, 1287, 1288, 1596, 1597, 1598, 2022, 2023, 2024, 2025, 2026, 2027, 2184, 2185, 2186, 33, 45, 69, 93, 129, 141, 165, 189, 213, 237, 261, 273, 285, 309, 345, 381, 393, 465, 477, 573, 574, 669, 717, 729, 753, 765, 777, 789, 801, 813, 849, 861, 885, 909, 921, 957, 981, 1017, 1029, 1053, 1089, 1101, 1113, 1149, 1161, 1173, 1197, 1221, 1245, 1269, 1281, 1305, 1329, 1341, 1342, 1377, 1413, 1437, 1449, 1473, 1509, 1533, 1569, 1617, 1629, 1701, 1725, 1821, 1822, 1857, 1917, 1953, 2013, 2109, 2205, 2229, 2265, 2289, 2301, 2313, 2325, 2349, 2361, 2373, 2385, 2397, 2409, 2433, 2445, 2457, 2469, 2481, 2493, 2505, 2517, 2529, 2541, 2553, 2565, 2577, 2589, 2601, 2613, 2637, 2649, 2661, 2673, 2685, 2686, 2709, 2721, 2733, 2745, 2757, 2769, 2781, 2793, 2805, 2829, 2841, 2853, 2865, 2877, 2889, 2901, 2913, 2925, 2937, 2949, 2961, 2973, 2985, 2997, 3009, 3033, 3045, 3057, 3069, 3081, 3093, 310

123.17200319415893

### Decompose flags

In [ ]:


tot = np.asarray([flux[i].shape[0] for i in oid_list]).mean()
#print(np.asarray([flux[i].shape[0] for i in oid_list]).std())
thruster = np.asarray([(flux[i].tpf_flags[flux[i].tpf_flags > 524287./2.]).count() for i in oid_list]).mean()
#thruster/tot
# cal=[]
# for i in oid_list:
#     c = 0
#     for x in flux[i].tpf_flags:
#         if 8192 in flagdecomp(int(x),decomp=[]):
#             c+=1
#     cal.append(c)
print(tot)

### Appendix, scrambled pieces

In [ ]:
location = 'stars_shortlist/{}/results/{}'.format(cluster,test)
maxlen, seq = generate_sequence(oid_list, location)


In [ ]:
overlap_dirty, overlap_clean, p, remove = remove_systematics(seq,maxlen,maxpoints=4)

In [ ]:
inflated_binoms = create_inflated_binoms(oid_list, p, maxlen,display=True)

In [ ]:
display_comparison(inflated_binoms, overlap_dirty, overlap_clean, test)